## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [158]:
import pandas as pd

df = pd.read_csv('client_dataset.csv')

#df.head()

In [159]:
# View the column names in the data
df.columns


Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [160]:
df.dtypes

first           object
last            object
job             object
phone           object
email           object
client_id        int64
order_id         int64
order_date      object
order_week       int64
order_year       int64
item_id         object
category        object
subcategory     object
unit_price     float64
unit_cost      float64
unit_weight    float64
qty              int64
line_number      int64
dtype: object

In [161]:
# Use the describe function to gather some basic statistics
df.describe()


,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [162]:
# Use this space to do any additional research
# and familiarize yourself with the data.
print(f" Frame shape {df.shape}")

 Frame shape (54639, 18)


In [163]:
# What three item categories had the most entries?
top3 = df["category"].value_counts()[:3]
print(top3) 

category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64


In [164]:
# For the category with the most entries,
# which subcategory had the most entries?
df.loc[df["category"] == "consumables"]["subcategory"].value_counts()[:1]


subcategory
bathroom supplies    6424
Name: count, dtype: int64

In [165]:
# Which five clients had the most entries in the data?
df["client_id"].value_counts().head()


client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [166]:
# Store the client ids of those top 5 clients in a list.
top5clients = list(df["client_id"].value_counts().keys()[:5])
print(top5clients)

[33615, 66037, 46820, 38378, 24741]


In [167]:
# How many total units (the qty column) did the
# client with the most entries order order?
totalqty = df.loc[df["client_id"] == top5clients[0]]["qty"].sum() 
print(f"Client: {top5clients[0]}")
print(f"Quantity: {totalqty}")

Client: 33615
Quantity: 64313


## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [168]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df["subtotal"] = df["unit_price"]*df["qty"]
df[["unit_price", "qty", "subtotal"]].head(2)

,unit_price,qty,subtotal
0,1096.80,105,115164.00
1,24.95,21,523.95


In [169]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
def shipping_price(row):
    orderweight = row["unit_weight"]*row["qty"]
    if orderweight > 50:
        price = 7*orderweight
        return price 
    price = orderweight * 10
    return price 

df["shipping_price"]=df.apply(shipping_price, axis=1)

In [170]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%
def total_price(row):
    totalcost = round((row["subtotal"]+row["shipping_price"])*1.0925,2)
    return totalcost 

df["line_price"] = df.apply(total_price, axis=1)

In [171]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).
def cost(row):
    cost = (row["unit_cost"]*row["qty"]+row["shipping_price"])
    return cost 

df["line_cost"] = df.apply(cost,axis=1)

In [172]:
# Create a column for the profit of
# each line using line cost and line price
def line_profit(row):
    totalprofit = row["line_price"]-row["line_cost"]
    return totalprofit 

df["line_profit"] = df.apply(line_profit,axis=1)


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [173]:
# Check your work using the totals above
total1 = round(df.loc[df["order_id"] == 2742071]["line_price"].sum(),2)
print(f"Order ID 2742071 total is {total1}")

total2 = round(df.loc[df["order_id"] == 2173913]["line_price"].sum(),2)
print(f"Order ID 2173913 total is {total2}")

total3 = round(df.loc[df["order_id"] == 6128929]["line_price"].sum(),2)
print(f"Order ID 6128929 total is {total3}")

Order ID 2742071 total is 152811.89
Order ID 2173913 total is 162388.71
Order ID 6128929 total is 923441.25


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [176]:
for client in top5clients:
    client_total = round(df.loc[df["client_id"] == client]["line_price"].sum(),2)
    print(f"Client {client} spent ${client_total}.")

Client 33615 spent $8377308.53.
Client 66037 spent $10259514.8.
Client 46820 spent $9743794.33.
Client 38378 spent $12906550.87.
Client 24741 spent $82268892.04.


In [180]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.
def find_total_price(client_id):
    return df.loc[df['client_id'] == client_id, 'line_price'].sum()

IDnumber = [33615, 66037, 46820, 38378, 24741]

for client_id in IDnumber:
    total_spent = find_total_price(client_id)
    print(f"{client_id}: ${total_spent:.2f}")

33615: $8377308.53
66037: $10259514.80
46820: $9743794.33
38378: $12906550.87
24741: $82268892.04


In [187]:
# Create a summary DataFrame showing the totals for the
# top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.

summary_data = []
for client in top5clients:
    clientQTY = df.loc[df["client_id"] == client]["qty"].sum()
    clientSHIP = round(df.loc[df["client_id"] == client]["shipping_price"].sum(),2)
    clientREV = round(df.loc[df["client_id"] == client]["line_price"].sum(),2)
    clientCOST = round(df.loc[df["client_id"] == client]["line_cost"].sum(),2)
    clientPROFIT = round(df.loc[df["client_id"] == client]["line_profit"].sum(),2)
    clientDATA = {"client_id": client,
                  "qty": clientQTY,
                  "total_shipping": clientSHIP,
                  "total_revenue": clientREV,
                  "total_cost": clientCOST,
                  "total_profit": clientPROFIT}
    summary_data.append(clientDATA)

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values("total_profit", ascending = False)
summary_df

,client_id,qty,total_shipping,total_revenue,total_cost,total_profit
4,24741,239862,5126448.37,82268892.04,45688899.71,36579992.33
3,38378,73667,3429455.40,12906550.87,9634720.98,3271829.89
1,66037,43018,1395151.85,10259514.80,7004482.98,3255031.82
2,46820,75768,1601448.84,9743794.33,7007191.64,2736602.69
0,33615,64313,1828984.89,8377308.53,6175313.91,2201994.62


In [282]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.

summary_df = summary_df.rename(columns={"client_id": "Client",
                            "qty": "Quantity",
                            "total_shipping":"Shipping(mil)",
                            "total_revenue":"Revenue(mil)",
                            "total_cost":"Cost(mil)",
                            "total_profit":"Profit(mil)"})

columns_to_scale = ["Shipping(mil)","Revenue(mil)", "Cost(mil)", "Profit(mil)"]
summary_df


,Client,Quantity,Shipping,Revenue,Cost,Profit
4,24741,239862,5126448.37,82268892.04,45688899.71,36579992.33
3,38378,73667,3429455.40,12906550.87,9634720.98,3271829.89
1,66037,43018,1395151.85,10259514.80,7004482.98,3255031.82
2,46820,75768,1601448.84,9743794.33,7007191.64,2736602.69
0,33615,64313,1828984.89,8377308.53,6175313.91,2201994.62


In [295]:
# Sort the updated data by "Total Profit" form highest to lowest
sorted_df = summary_df.sort_values(by="Profit", ascending=False)
sorted_df

,Client,Quantity,Shipping,Revenue,Cost,Profit
4,24741,239862,5126448.37,82268892.04,45688899.71,36579992.33
3,38378,73667,3429455.40,12906550.87,9634720.98,3271829.89
1,66037,43018,1395151.85,10259514.80,7004482.98,3255031.82
2,46820,75768,1601448.84,9743794.33,7007191.64,2736602.69
0,33615,64313,1828984.89,8377308.53,6175313.91,2201994.62


In [ ]:
##Write a brief 2-3 sentence summary of your findings:

#The 'client_dataset.csv' is a 54639 row 18 column list outlining bulk purchases over time.
#I transformed the data for analysis and made it more readable in a data frame, calculated shipping costs,
#taxes, and profit from sale. Finally, renamed and organized data frame by highest cost for presentation. 
